# Fast text and subwords embeddings

This notebook explore the embeddings of words according to their sub-words structure.
Subwords models capture the similarity of words according to their roots and composition.
In that sense, ``theology`` will be close to ``geology``.

## More about..
https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py :

The main principle behind fastText is that the morphological structure of a word carries important information about the meaning of the word. Such structure is not taken into account by traditional word embeddings like Word2Vec, which train a unique word embedding for every individual word.

### FastText
fastText attempts to solve this by treating each word as the aggregation of its subwords. For the sake of simplicity and language-independence, subwords are taken to be the character ngrams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.

### FastText or Word2Vec?
- Word2vec > FasText on semantic tasks
- TasText > Word2Vec on syntactic tasks
The difference goes smaller as the size of the training set increases.

### Main advantage:
fastText can obtain vectors even for out-of-vocabulary (OOV) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.



In [1]:
from bpemb import BPEmb

# Load english model

In [2]:
bpemb_en = BPEmb(lang="en")
bpemb_en.encode("Stratford")
bpemb_en.encode("This is anarchism")


/home/bettyld/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


['▁this', '▁is', '▁an', 'arch', 'ism']

# Get embedding vectors

In [3]:
print(type(bpemb_en.emb))
print(bpemb_en.vectors.shape)

<class 'gensim.models.keyedvectors.KeyedVectors'>
(10000, 100)


# find unknown similar words

In [4]:
bpemb_en.most_similar("osis", topn=20)

[('▁disease', 0.8304932117462158),
 ('▁diagn', 0.8269731998443604),
 ('itis', 0.7845749855041504),
 ('▁patients', 0.7690582871437073),
 ('▁cancer', 0.7671700119972229),
 ('▁symptoms', 0.7551704049110413),
 ('orders', 0.7446448802947998),
 ('▁syndrome', 0.7400376796722412),
 ('▁hyp', 0.7319927215576172),
 ('▁treatment', 0.7303920388221741),
 ('▁diseases', 0.7287130951881409),
 ('▁chronic', 0.7265974283218384),
 ('ysis', 0.7166935205459595),
 ('▁tum', 0.7108365297317505),
 ('▁neuro', 0.708076000213623),
 ('▁inf', 0.7016445994377136),
 ('▁sympt', 0.6954596638679504),
 ('▁disorder', 0.6911265254020691),
 ('inal', 0.6867300271987915),
 ('▁surgery', 0.6753949522972107)]

# Fasttext

In [5]:
from gensim.models import FastText
from gensim.test.utils import common_texts  # some example sentences

In [9]:
print(common_texts[0])
model = FastText(vector_size=4, window=3, min_count=1)  # instantiate
model.build_vocab(common_texts)
model.train(corpus_iterable=common_texts, total_examples=len(common_texts), epochs=10)  # train

['human', 'interface', 'computer']


(36, 290)

In [11]:
model2 = FastText(vector_size=4, window=3, min_count=1, sentences=common_texts, epochs=10)

In [12]:
import numpy as np

np.allclose(model.wv['computer'], model2.wv['computer'])


True

# Create embedding for non-existent words!

In [14]:
# existent word
existent_word = "computer"
print(existent_word in model.wv.key_to_index)
computer_vec = model.wv[existent_word]  # numpy vector of a word

# Non-existent word
oov_word = "graph-out-of-vocab"
print(oov_word in model.wv.key_to_index)
oov_vec = model.wv[oov_word]  # numpy vector for OOV word

print(computer_vec, oov_vec)

True
False
[-0.00263242 -0.03817195 -0.02007434  0.00741351] [-0.02574481 -0.01075656  0.00200511  0.03659087]


# Byte Pair Encoding


Byte pair encoding performs a statistical analysis of the training dataset to discover common symbols within a word, such as consecutive characters of arbitrary length. Starting from symbols of length  1 , byte pair encoding iteratively merges the most frequent pair of consecutive symbols to produce new longer symbols. 

[sce](https://d2l.ai/chapter_natural-language-processing-pretraining/subword-embedding.html) 

In [38]:
import collections

symbols = [
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
    'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]']

In [39]:
# start with single character frequency

raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

In [40]:
def get_max_freq_pair(token_freqs):
    """Returns the most frequent pair of consecutive
    symbols within a word,
    where words are the keys of argument ``token_freqs``"""
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # Key of `pairs` is a tuple of two consecutive symbols
            pairs[symbols[i], symbols[i + 1]] += freq
    print('pairs ends:', pairs)
    return max(pairs, key=pairs.get)  # Key of `pairs` with the max value
max_freq_pair = get_max_freq_pair(token_freqs)
max_freq_pair

pairs ends: defaultdict(<class 'int'>, {('f', 'a'): 7, ('a', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'): 7, ('r', '_'): 7, ('t', 'a'): 9, ('a', 'l'): 9, ('l', 'l'): 9, ('l', '_'): 5, ('l', 'e'): 4})


('t', 'a')

In [41]:
# we merge the characters with the highest frequency as the same character
def merge_symbols(max_freq_pair, token_freqs, symbols):
    """merge the most frequent pair of consecutive symbols
    to produce new symbols"""
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs
merge_symbols(max_freq_pair, token_freqs, symbols)

{'f a s t _': 4, 'f a s t e r _': 3, 'ta l l _': 5, 'ta l l e r _': 4}

### Process byte pair encoding iteratively.

First iteration the most frequent pair of consecutive symbols are 't' and 'a'.

Second iteration, the most frequent pair of consecutive symbols are 'f' and 'a'.

In [42]:
# Process byte pair encoding iteratively
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'merge #{i + 1}:', max_freq_pair)

pairs ends: defaultdict(<class 'int'>, {('f', 'a'): 7, ('a', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'): 7, ('r', '_'): 7, ('t', 'a'): 9, ('a', 'l'): 9, ('l', 'l'): 9, ('l', '_'): 5, ('l', 'e'): 4})
merge #1: ('t', 'a')
pairs ends: defaultdict(<class 'int'>, {('f', 'a'): 7, ('a', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'): 7, ('r', '_'): 7, ('ta', 'l'): 9, ('l', 'l'): 9, ('l', '_'): 5, ('l', 'e'): 4})
merge #2: ('ta', 'l')
pairs ends: defaultdict(<class 'int'>, {('f', 'a'): 7, ('a', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'): 7, ('r', '_'): 7, ('tal', 'l'): 9, ('l', '_'): 5, ('l', 'e'): 4})
merge #3: ('tal', 'l')
pairs ends: defaultdict(<class 'int'>, {('f', 'a'): 7, ('a', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'): 7, ('r', '_'): 7, ('tall', '_'): 5, ('tall', 'e'): 4})
merge #4: ('f', 'a')
pairs ends: defaultdict(<class 'int'>, {('fa', 's'): 7, ('s', 't'): 7, ('t', '_'): 4, ('t', 'e'): 3, ('e', 'r'

After 10 iterations of byte pair encoding, we can see that list symbols now contains 10 more symbols that are iteratively merged from other symbols.

In [43]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


In [44]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']
